# AI Model Training - Text Classification

This notebook demonstrates training custom AI models for the AI-Powered BaaS platform.
We'll create a text classification model for sentiment analysis.

In [ ]:
# Install required packages
!pip install torch transformers scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import json
from datetime import datetime

## 1. Data Preparation

Load and prepare training data for sentiment analysis.

In [ ]:
# Sample training data - in production, you'd load from your database
sample_data = [
    {"text": "I love this product! It's amazing.", "label": 1},
    {"text": "This is terrible, I hate it.", "label": 0},
    {"text": "Pretty good overall, would recommend.", "label": 1},
    {"text": "Not worth the money, disappointing.", "label": 0},
    {"text": "Excellent quality and great service!", "label": 1},
    {"text": "Worst experience ever, avoid at all costs.", "label": 0},
    {"text": "Satisfied with the purchase, good value.", "label": 1},
    {"text": "Poor quality, broke within a week.", "label": 0},
    {"text": "Outstanding product, exceeded expectations!", "label": 1},
    {"text": "Complete waste of time and money.", "label": 0}
]

# Create DataFrame
df = pd.DataFrame(sample_data)
print(f"Dataset shape: {df.shape}")
print(f"Label distribution:\n{df['label'].value_counts()}")
df.head()

## 2. Model Setup

Initialize the pre-trained model and tokenizer.

In [ ]:
# Model configuration
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2
MAX_LENGTH = 512

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=NUM_LABELS
)

print(f"Model: {MODEL_NAME}")
print(f"Number of parameters: {model.num_parameters():,}")

In [ ]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH
    )

# Prepare datasets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# Create dataset class
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

## 3. Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
)

# Metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions)
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Training configuration complete!")

## 4. Model Training

In [ ]:
# Start training
print("Starting training...")
training_start = datetime.now()

trainer.train()

training_end = datetime.now()
training_duration = training_end - training_start
print(f"Training completed in: {training_duration}")

## 5. Model Evaluation

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

In [ ]:
# Test predictions
def predict_sentiment(text):
    model.eval()
    encoding = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = model(**encoding)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        confidence = torch.max(predictions).item()
        predicted_class = torch.argmax(predictions, dim=-1).item()
        
    return {
        'sentiment': 'positive' if predicted_class == 1 else 'negative',
        'confidence': confidence,
        'raw_predictions': predictions.numpy()[0]
    }

# Test examples
test_texts = [
    "This is absolutely fantastic!",
    "I'm really disappointed with this.",
    "It's okay, nothing special.",
    "Best purchase I've ever made!",
    "Completely useless product."
]

print("Test Predictions:")
for text in test_texts:
    result = predict_sentiment(text)
    print(f"Text: '{text}'")
    print(f"Sentiment: {result['sentiment']} (confidence: {result['confidence']:.3f})")
    print("-" * 50)

## 6. Model Export and Deployment

In [ ]:
# Save the model
model_save_path = "./custom_sentiment_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Create model metadata
model_metadata = {
    "model_name": "custom-sentiment-classifier",
    "base_model": MODEL_NAME,
    "task": "text-classification",
    "labels": ["negative", "positive"],
    "training_date": datetime.now().isoformat(),
    "training_duration": str(training_duration),
    "eval_accuracy": eval_results["eval_accuracy"],
    "model_size": f"{model.num_parameters():,} parameters",
    "max_length": MAX_LENGTH,
    "training_samples": len(train_dataset),
    "validation_samples": len(val_dataset)
}

# Save metadata
with open(f"{model_save_path}/model_metadata.json", "w") as f:
    json.dump(model_metadata, f, indent=2)

print(f"Model saved to: {model_save_path}")
print("Model Metadata:")
print(json.dumps(model_metadata, indent=2))

## 7. Integration with AI-Powered BaaS

Code to integrate the trained model with the BaaS platform.

In [ ]:
# Example integration code for the AI service
integration_code = '''
# Add this to your AI service (ai-services/main.py)

class CustomSentimentModel:
    def __init__(self, model_path="./custom_sentiment_model"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.eval()
    
    def predict(self, text: str) -> dict:
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            outputs = self.model(**encoding)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            confidence = torch.max(predictions).item()
            predicted_class = torch.argmax(predictions, dim=-1).item()
        
        return {
            "sentiment": "positive" if predicted_class == 1 else "negative",
            "confidence": confidence,
            "scores": {
                "negative": predictions[0][0].item(),
                "positive": predictions[0][1].item()
            }
        }

# Initialize the custom model
custom_sentiment_model = CustomSentimentModel()

# Add endpoint
@app.post("/ai/custom/sentiment")
async def custom_sentiment_analysis(request: SentimentAnalysisRequest):
    start_time = datetime.utcnow()
    
    try:
        result = custom_sentiment_model.predict(request.text)
        processing_time = (datetime.utcnow() - start_time).total_seconds() * 1000
        
        return AIResponse(
            success=True,
            data=result,
            timestamp=datetime.utcnow(),
            model_used="custom-sentiment-classifier",
            processing_time_ms=processing_time
        )
    except Exception as e:
        return AIResponse(
            success=False,
            error=str(e),
            timestamp=datetime.utcnow(),
            model_used="custom-sentiment-classifier",
            processing_time_ms=(datetime.utcnow() - start_time).total_seconds() * 1000
        )
'''

print("Integration code:")
print(integration_code)

## 8. Performance Monitoring

Set up monitoring for the deployed model.

In [ ]:
# Create monitoring dashboard
monitoring_code = '''
# Model performance monitoring
import time
from collections import defaultdict

class ModelMonitor:
    def __init__(self):
        self.prediction_count = 0
        self.total_processing_time = 0
        self.confidence_scores = []
        self.sentiment_distribution = defaultdict(int)
        
    def log_prediction(self, result: dict, processing_time: float):
        self.prediction_count += 1
        self.total_processing_time += processing_time
        self.confidence_scores.append(result["confidence"])
        self.sentiment_distribution[result["sentiment"]] += 1
    
    def get_stats(self):
        if self.prediction_count == 0:
            return {"error": "No predictions logged yet"}
            
        return {
            "total_predictions": self.prediction_count,
            "avg_processing_time_ms": self.total_processing_time / self.prediction_count,
            "avg_confidence": sum(self.confidence_scores) / len(self.confidence_scores),
            "sentiment_distribution": dict(self.sentiment_distribution)
        }

model_monitor = ModelMonitor()
'''

print("Model monitoring setup:")
print(monitoring_code)

## Summary

This notebook demonstrated:

1. **Data Preparation**: Loading and preprocessing training data
2. **Model Setup**: Initializing a pre-trained transformer model
3. **Training**: Fine-tuning the model on custom data
4. **Evaluation**: Testing model performance
5. **Export**: Saving the model for deployment
6. **Integration**: Code for integrating with the AI-Powered BaaS platform
7. **Monitoring**: Setting up performance monitoring

The trained model can now be deployed as part of the AI-Powered BaaS platform, providing custom sentiment analysis capabilities alongside the Gemini API integration.